### User mode to kernel mode

Often called entering the kernel. Three methods: 
1. Exceptions
2. Interrupts (asynchronous exceptions, asynchronous, because the cause is outside out control)
3. System calls/Traps    
  
OS saves state of user program, hardware identifies why boundary was crossed (one of the three methods above), hardware selects entry from exception table, approproate handler is invoked.  
One interrupt stack per processor/process/thread

System call is a request by a user level process to call a function in the kernel.  
Is part of the interface between the application and the operating system.  
1. User process executes a trap/system call instruction
2. Hardware calls the OS at the system call handler, at a pre-specified location
3. OS then identifies the required service and parameters, executes the device, sets register to contain result, returns result to user perogram
4. User [program receives the result and continues to next instruction.  
  
From an interrupt, just reverse all steps. PC needs to be incremented on return from exception and system call. This is because these are synchronous, so you want to execute the next instruction, not the same one. On exception, the handler 

Dual Mode Execution is simply one piece of the protection pie. For efficient protection, the hardware must support at least three features:
1. Priveleged Instructions, Instructions only available in Kernel mode. In user mode, there should be no way to execut potentially unsafe instructions, prevents the user process from running syscalls whenever they want to, and the implementation is a mode bit.  
2. Timer Interrupts, allow kernel to regain control from the process, and prevents process from gaining control of the CPU and never releasing it. Implementation: Hardware timer can be set to expire after a delay and pass control back to the kernel.
3. Memory protection, In user mode, memory access outside a process memory is prohibited, prevents unauthorized access of data

## Processes

A process is a program during execution.  
Program = static file.   
Process = executing program = program + execution state.  
A process is a basiic unit of execution in an OS.  
Different processes may run different instances of the same program.  
Process execution requires memory to contain the program code and data.  

Process state consists of at least the code for running the program, the PC indicating the next instruction, an execution stack with the program's call chain and stack pointer, static data for running the program, space for dynamic data (the heap), and the heap pointer, values of CPU registers, a set of OS resouces in use, PID, process execution state

Processes are either running, ready, blocked, new, or terminated.
1. New: OS is setting up the process state
2. Ready: Process is ready but waiting for the CPU
3. Blocked: Waiting for an event to complete
4. Running: Executing instructions on the CPU
5. Terminating: OS is destroying this process

The OS tracks this data using a PCB (Process Control Block).  
Dynamic kernel data structure kept in memory, represents the execution state and location of each process when it is not executing.  
PCB containts, PID, PC, stack pointer, contents of registers, memory info, basically any process state not stored in address space.  
PCBs are initialized when a process is created and deleted when a process terminates.

When a program begins running, the lodaer reads and interprets the executable file, sets up the process memory to contain the code and data from executable, pushes argc and argv onto the stack, and then sets the CPU registers and calls start().  
When main returns OS calls exit, which destroys the process and returns all resources, unless the process calls exit on its own

Several ways to create processes: One process can create other processes, parent defines resources and priveleges to children. Parent can either wait for the child to complete or continue in parallel

Processes are created by fork()   
Fork() copies process into an identical process, copies variables and program counter from parent to child    
return valye is different for parent and child, for parent it is the parent ID, for child it is 0  
Both processes begin execution from same point, immediately after fork   
Each process has its own memory and its own copy of each variable.
Changes to variables in one process are not reflected in the other process

exec() overlays a process with a new program.  
PID does not change, arguments to program may be specified.  
Code, stack, and heap are overwritten.  
Child processes call exec() to start a new, different program.  
If call is successful, it is in the same process, but it is running a different program

wait() syscall causes a parent process to wait for a child process to terminate.  
It allows a parent process to get the return value from the child  
Puts the parent to sleep waiting for the child's result   
When a child calls exit(), the OS unblocks the parent and returns the value passed by exit() as a result of the wait call  
If there are no children, wait() returns immediately.  
If there are zombies, wait returns one of the values immediately
Zombie process- process has terminated, but parent process has not collected its status

After the program finishes execution, it calls exit().  
Takes the result of the pgoram as an argument, closes all open files, deallocates memory and OS structures supporting process.  
If parent is alive, holds the result value, until the parent requests it, which makes it technically a zombie state.  
Otherwise, it deallocates all data structures, process is dead.  
Cleans up all waiting zombies.

kill() terminates a process.   
Sends a signal to a specified process. Receiving process can define signal handlers to handle signals in a particular way.  
If there are no signal handlers, the default action is taken.

Orphaned processes: parent terminates before the child, child can orphan itself to keep running in the background

OS must include calls to enable special control of a process:  
Priority manip: nice(), specifies the base process priority.  
ptrace() allows a process to be put under the control of another process.  
Sleep puts a process on timer queue waiting for some number of seconds